In [46]:
  !git clone -q https://github.com/fsiraj/Vegetation-Encroachment-on-Power-Infrastructure
 !mv ./Vegetation-Encroachment-on-Power-Infrastructure/* ./
  !rm -rf sample_data
  !rm -rf Vegetation-Encroachment-on-Power-Infrastructure

IndentationError: ignored

In [2]:
 !pip install -q geopandas geemap

     |████████████████████████████████| 994 kB 30.8 MB/s 
     |████████████████████████████████| 476 kB 48.1 MB/s 
     |████████████████████████████████| 6.6 MB 47.5 MB/s 
     |████████████████████████████████| 15.4 MB 37 kB/s 
     |████████████████████████████████| 130 kB 62.3 MB/s 
     |████████████████████████████████| 1.6 MB 62.2 MB/s 
     |████████████████████████████████| 98 kB 8.0 MB/s 
     |████████████████████████████████| 76 kB 5.7 MB/s 
     |████████████████████████████████| 94 kB 3.6 MB/s 
     |████████████████████████████████| 1.2 MB 26.9 MB/s 
     |████████████████████████████████| 3.3 MB 49.6 MB/s 
     |████████████████████████████████| 1.3 MB 54.0 MB/s 
     |████████████████████████████████| 219 kB 74.1 MB/s 
     |████████████████████████████████| 76 kB 5.0 MB/s 
     |████████████████████████████████| 115 kB 73.9 MB/s 
     |████████████████████████████████| 112 kB 59.7 MB/s 
     |████████████████████████████████| 392 kB 44.6 MB/s 
     |█████████████████

In [3]:
import ee
import geemap
import geopandas as gpd
import pandas as pd
import datetime

In [4]:
 ee.Authenticate()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=KfPk4NUmETEuCyI5bnXAuEZkvHHwvWVq11aaw3MxaK8&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AX4XfWhWbfOpwI8RSR1Wka6wqdnxJe3mmMXXY4OpW4clYAIW9uPVLYVWSFo

Successfully saved authorization token.


In [5]:
ee.Initialize()

In [7]:
saoMiguel = ee.Geometry.Point([-25.3425, 37.7532])
boundary = geemap.kml_to_ee('./study_area.kml').geometry()

# Cloud Masking

## Helpers

In [27]:
def add_cloud_bands(img):
    # Get s2cloudless image, subset the probability band.
    cld_prb = ee.Image(img.get('s2cloudless')).select('probability')

    # Condition s2cloudless by the probability threshold value.
    is_cloud = cld_prb.gt(CLD_PRB_THRESH).rename('clouds')

    # Add the cloud probability layer and cloud mask as image bands.
    return img.addBands(ee.Image([cld_prb, is_cloud]))

def add_shadow_bands(img):
    # Identify water pixels from the SCL band.
    not_water = img.select('SCL').neq(6)

    # Identify dark NIR pixels that are not water (potential cloud shadow pixels).
    SR_BAND_SCALE = 1e4
    dark_pixels = img.select('B8').lt(NIR_DRK_THRESH*SR_BAND_SCALE).multiply(not_water).rename('dark_pixels')

    # Determine the direction to project cloud shadow from clouds (assumes UTM projection).
    shadow_azimuth = ee.Number(90).subtract(ee.Number(img.get('MEAN_SOLAR_AZIMUTH_ANGLE')));

    # Project shadows from clouds for the distance specified by the CLD_PRJ_DIST input.
    cld_proj = (img.select('clouds').directionalDistanceTransform(shadow_azimuth, CLD_PRJ_DIST*10)
        .reproject(**{'crs': img.select(0).projection(), 'scale': 100})
        .select('distance')
        .mask()
        .rename('cloud_transform'))

    # Identify the intersection of dark pixels with cloud shadow projection.
    shadows = cld_proj.multiply(dark_pixels).rename('shadows')

    # Add dark pixels, cloud projection, and identified shadows as image bands.
    return img.addBands(ee.Image([dark_pixels, cld_proj, shadows]))

def add_cld_shdw_mask(img):
    # Add cloud component bands.
    img_cloud = add_cloud_bands(img)

    # Add cloud shadow component bands.
    img_cloud_shadow = add_shadow_bands(img_cloud)

    # Combine cloud and shadow mask, set cloud and shadow as value 1, else 0.
    is_cld_shdw = img_cloud_shadow.select('clouds').add(img_cloud_shadow.select('shadows')).gt(0)

    # Remove small cloud-shadow patches and dilate remaining pixels by BUFFER input.
    # 20 m scale is for speed, and assumes clouds don't require 10 m precision.
    is_cld_shdw = (is_cld_shdw.focal_min(2).focal_max(BUFFER*2/20)
        .reproject(**{'crs': img.select([0]).projection(), 'scale': 20})
        .rename('cloudmask'))

    # Add the final cloud-shadow mask to the image.
    return img_cloud_shadow.addBands(is_cld_shdw)

def apply_cld_shdw_mask(img):
    # Subset the cloudmask band and invert it so clouds/shadow are 0, else 1.
    not_cld_shdw = img.select('cloudmask').Not()

    # Subset reflectance bands and update their masks, return the result.
    return img.select('B.*').updateMask(not_cld_shdw)

## Parameters

In [28]:
AOI = saoMiguel
START_DATE = '2018-01-01'
END_DATE = '2021-07-30'
CLOUD_FILTER = 60
CLD_PRB_THRESH = 60
NIR_DRK_THRESH = 0.15
CLD_PRJ_DIST = 2
BUFFER = 10

## Build Sentinel-2 Collection

In [29]:
def get_s2_sr_cld_col(aoi, start_date, end_date):
    # Import and filter S2 SR.
    s2_sr_col = (ee.ImageCollection('COPERNICUS/S2_SR')
        .filterBounds(aoi)
        .filterDate(start_date, end_date))

    # Import and filter s2cloudless.
    s2_cloudless_col = (ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY')
        .filterBounds(aoi)
        .filterDate(start_date, end_date))

    # Join the filtered s2cloudless collection to the SR collection by the 'system:index' property.
    return ee.ImageCollection(ee.Join.saveFirst('s2cloudless').apply(**{
        'primary': s2_sr_col,
        'secondary': s2_cloudless_col,
        'condition': ee.Filter.equals(**{
            'leftField': 'system:index',
            'rightField': 'system:index'
        })
    }))

In [30]:
sen2 = get_s2_sr_cld_col(AOI, START_DATE, END_DATE)

## Apply cloud masks

In [31]:
sen2 = sen2.map(add_cld_shdw_mask).map(apply_cld_shdw_mask)

# Visualizations

## Helpers

In [32]:
def visualize(collection, dates, time_delta, ndvi=False, ndwi=False):
  # Create the map
  Map = geemap.Map(center=[37.7529, -25.3453], zoom=13)
  Map.add_kml('study_area.kml',layer_name='Study Area')
  # Add the images as layers
  delta = pd.Timedelta(days=time_delta)
  for date in dates:
    print(f"\rProcessing {date.date()}", end='')
    image = ee.Image(collection
                    .filterDate(date, date + delta)
                    .median()
                    .clip(boundary)
                    )
    layer_name = str(date).split(' ')[0]
    Map.addLayer(image, trueColour, layer_name, shown=False)
    if ndvi:
      Map.addLayer(image, ndvi_params, layer_name + ' NDVI', shown=False)
    if ndwi:
      Map.addLayer(image, ndwi_params, layer_name + ' NDWI', shown=False)

  Map.addLayerControl()
  return Map

## Parameters

In [33]:
trueColour = {
    'bands': ["B4", "B3", "B2"],
    'min': 0,
    'max': 3000
}

ndvi_params = {
    'bands'   : ['ndvi'],
    'min'     : -1,
    'max'     : 1,
    'palette' : ['blue', 'white', 'green']
}

ndwi_params = {
    'bands'   : ['ndwi'],
    'min'     : -1,
    'max'     : 1,
    'palette': ['00FFFF', '0000FF']
}





## Proposed Dataset
The proposed dataset still contains too much cloud cover, will try with monthly composites instead

In [34]:
dates = pd.read_excel('./proposed_dataset_list.xlsx', usecols=['DATE'], parse_dates=True)['DATE'].sort_values()
dates

33   2018-01-16
81   2018-02-02
34   2018-02-10
35   2018-02-25
36   2018-03-02
        ...    
69   2021-07-01
4    2021-07-04
6    2021-07-14
5    2021-07-14
70   2021-07-21
Name: DATE, Length: 83, dtype: datetime64[ns]

In [35]:
print(f"Start: {dates.iloc[0].date()}, End: {dates.iloc[-1].date()}")
dates.iloc[-1] - dates.iloc[0]

Start: 2018-01-16, End: 2021-07-21


Timedelta('1282 days 00:00:00')

In [36]:
Map = geemap.Map(center=[37.7529, -25.3453], zoom=13)
Map.add_kml('study_area.kml',layer_name='Study Area')

delta = pd.Timedelta(days=1)
for date in dates:
  print(f"\rProcessing {date.date()}", end='')
  image = ee.Image(sen2
                   .filterDate(date - delta, date + delta)
                   .sort("CLOUD_COVERAGE_ASSESSMENT")
                   .first()
                   .clip(boundary)
                   )
  layer_name = str(date).split(' ')[0]
  Map.addLayer(image, trueColour, layer_name, shown=False)

Map.addLayerControl()
Map

Processing 2021-07-21

## Bi-weekly Composites

In [37]:
dates_BW = pd.date_range("2018-01-01", "2021-08-01", freq="SM").to_list()
len(dates_BW)

86

In [38]:
visualize(sen2, dates_BW, time_delta=15)

Processing 2021-07-31

## Monthly Composites

In [39]:
dates_M = pd.date_range("2018-01-01", "2021-08-01", freq="MS").to_list()
len(dates_M)

44

In [40]:
visualize(sen2, dates_M, time_delta=30)

Processing 2021-08-01

## Quarterly

In [41]:
dates_Q = pd.date_range("2018-01-01", "2021-08-01", freq="QS").to_list()
len(dates_Q)

15

In [42]:
visualize(sen2, dates_Q, time_delta=90)

Processing 2021-07-01

## Normalized Difference Vegetation Index (NDVI) and Normalized Difference Water Index (NDWI)

### Calculate and add NDVI and NDWI bands

In [43]:
def add_ndvi(image):
  return image.addBands(image.normalizedDifference(['B8', 'B4']).rename('ndvi'))

def add_ndwi(image):
  return image.addBands(image.normalizedDifference(['B3', 'B8']).rename('ndwi'))

In [44]:
sen2_bands = sen2.map(add_ndvi).map(add_ndwi)

sen2_bands

In [45]:
visualize(sen2_bands, dates_Q, time_delta=90, ndvi=True, ndwi=True)

Processing 2021-07-01